In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

PRESET = os.environ.setdefault("PRESET", "test_spu_aggregator")
SELF_PARTY = os.environ.setdefault("SELF_PARTY", "sim")
os.environ["NODE_ENV"] = "development"

In [ ]:
from pathlib import Path
import ray
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation.sdk import (
    create_profiler,
    inherit_tracing_context,
    setup_tracing,
)

ray.shutdown()

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:{SELF_PARTY}"

setup_tracing()
# setup_debug_exporter()


with trace.get_tracer(__name__).start_as_current_span(
    "sim_trace",
    context=inherit_tracing_context(),
):
    with create_profiler() as profiler:
        with open(f"./presets/{PRESET}/_world.py") as f:
            _world = compile(f.read(), str(Path(f.name).resolve()), "exec")
        with open(f"./presets/{PRESET}/_algorithm.py") as f:
            _algorithm = compile(f.read(), str(Path(f.name).resolve()), "exec")
        exec(_world, globals())
        exec(_algorithm, globals())

In [ ]:
from secretnote.display import visualize_run

visualize_run(profiler)